#### 특정 웹툰 페이지의 모든 image를 다운로드 받기
* img 폴더를 생성하고 그 아래에 파일을 저장하기
* pathlib 의 Path 사용하여 디렉토리 생성하기

In [18]:
import requests
from bs4 import BeautifulSoup
from pathlib import Path

def download_one_episode(title,no,url):
    
    req_header = {'referer': url}
    
    res = requests.get(url)
    print(res.ok)
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
               
        img_tags = soup.select("img[src*='IMAG01']")
        # list comprehension 사용
        imgurl_list = [img['src'] for img in img_tags]
        print(len(imgurl_list))
        
        save_dir = Path('img') / title / str(no)
        save_dir.mkdir(parents=True, exist_ok=True)

        for idx,img_url in enumerate(imgurl_list,1):
            res = requests.get(img_url,headers=req_header)
            res.raise_for_status()

            file_name = Path(img_url).name
            save_path = save_dir / file_name
            save_path.write_bytes(res.content)
            print(f'다운로드 완료: {save_path} ({save_path.stat().st_size:,} bytes)')
                
if __name__ == '__main__':                
    download_one_episode('어린이집 다니는 구나',130,'https://comic.naver.com/webtoon/detail?titleId=817945&no=130&week=wed')

True
17
다운로드 완료: img\어린이집 다니는 구나\130\20250328181328_999f3cac3a98f930a659f6d70babda21_IMAG01_1.jpg (130,189 bytes)
다운로드 완료: img\어린이집 다니는 구나\130\20250328181328_999f3cac3a98f930a659f6d70babda21_IMAG01_2.jpg (151,109 bytes)
다운로드 완료: img\어린이집 다니는 구나\130\20250328181328_999f3cac3a98f930a659f6d70babda21_IMAG01_3.jpg (145,483 bytes)
다운로드 완료: img\어린이집 다니는 구나\130\20250328181328_999f3cac3a98f930a659f6d70babda21_IMAG01_4.jpg (150,147 bytes)
다운로드 완료: img\어린이집 다니는 구나\130\20250328181328_999f3cac3a98f930a659f6d70babda21_IMAG01_5.jpg (199,050 bytes)
다운로드 완료: img\어린이집 다니는 구나\130\20250328181328_999f3cac3a98f930a659f6d70babda21_IMAG01_6.jpg (184,634 bytes)
다운로드 완료: img\어린이집 다니는 구나\130\20250328181328_999f3cac3a98f930a659f6d70babda21_IMAG01_7.jpg (151,225 bytes)
다운로드 완료: img\어린이집 다니는 구나\130\20250328181328_999f3cac3a98f930a659f6d70babda21_IMAG01_8.jpg (175,616 bytes)
다운로드 완료: img\어린이집 다니는 구나\130\20250328181328_999f3cac3a98f930a659f6d70babda21_IMAG01_9.jpg (118,270 bytes)
다운로드 완료: img\어린이집 다니는 구나\130\202503281

#### 네이버 웹툰 여러 개 회차 Image 다운로드

In [ ]:
import requests
from urllib.parse import urlparse, parse_qs

def download_all_episode(title,episode_url):
    parsed_url = urlparse(episode_url)
    query_params = parse_qs(parsed_url.query)
    title_id = query_params.get('titleId', [''])[0]
    
    api_url = f'https://comic.naver.com/api/article/list?titleId={title_id}'
               #https://comic.naver.com/api/article/list?titleId=817945&page=1
    res = requests.get(api_url)
    print(res.status_code)    
    if res.ok: 
        for article in res.json()['articleList']:
            no = article['no']
            detail_url = f'https://comic.naver.com/webtoon/detail?titleId={title_id}&no={no}'
            print(detail_url)
        

if __name__ == '__main__': 
    download_all_episode('어린이집 다니는 구나','https://comic.naver.com/webtoon/list?titleId=817945')

200
https://comic.naver.com/webtoon/detail?titleId=817945&no=133
https://comic.naver.com/webtoon/detail?titleId=817945&no=132
https://comic.naver.com/webtoon/detail?titleId=817945&no=131
https://comic.naver.com/webtoon/detail?titleId=817945&no=130
https://comic.naver.com/webtoon/detail?titleId=817945&no=129
https://comic.naver.com/webtoon/detail?titleId=817945&no=128
https://comic.naver.com/webtoon/detail?titleId=817945&no=127
https://comic.naver.com/webtoon/detail?titleId=817945&no=126
https://comic.naver.com/webtoon/detail?titleId=817945&no=125
https://comic.naver.com/webtoon/detail?titleId=817945&no=124
https://comic.naver.com/webtoon/detail?titleId=817945&no=123
https://comic.naver.com/webtoon/detail?titleId=817945&no=122
https://comic.naver.com/webtoon/detail?titleId=817945&no=121
https://comic.naver.com/webtoon/detail?titleId=817945&no=120
https://comic.naver.com/webtoon/detail?titleId=817945&no=119
https://comic.naver.com/webtoon/detail?titleId=817945&no=118
https://comic.naver.

In [ ]:
from urllib.parse import urlparse, parse_qs

"""url에 titleId를 반환하는 함수"""
def get_title_id(url):
    parsed_url = urlparse(url)
    query_params = parse_qs(parsed_url.query)
    title_id = query_params.get('titleId', [''])[0]
    return title_id

#테스트 하기
url = 'https://comic.naver.com/webtoon/list?titleId=817945'
print(get_title_id(url))  # 출력: 817945

817945


In [ ]:
def calculate_pages(total_items, items_per_page=20):
    """총 페이지 수 계산 함수"""
    return (total_items + items_per_page - 1) // items_per_page

# 예제 사용
total_items = 138
items_per_page = 20

total_pages = calculate_pages(total_items)
print(f"총 {total_items}개의 항목을 {items_per_page}개씩 출력할 때 필요한 페이지 수: {total_pages}")
# 출력: 총 138개의 항목을 20개씩 출력할 때 필요한 페이지 수: 7

총 138개의 항목을 20개씩 출력할 때 필요한 페이지 수: 7


In [27]:
import requests

def download_all_episode(title,episode_url):
    title_id = get_title_id(episode_url)

    ajax_url = f'https://comic.naver.com/api/article/list?titleId={title_id}'               
    res = requests.get(ajax_url)

    if res.ok:
        total_count = res.json()['totalCount']
        for count in range(calculate_pages(total_count)):
            # count 변수 0,1,2 // page 번호는 1,2,3
            page = count + 1
            req_param = { "page": page }
            print(req_param)
            res = requests.get(ajax_url, params=req_param)
            for article in res.json()['articleList']:
                no = article['no']
                detail_url = f'https://comic.naver.com/webtoon/detail?titleId={title_id}&no={no}'
                print(detail_url)
        

if __name__ == '__main__': 
    download_all_episode('어린이집 다니는 구나','https://comic.naver.com/webtoon/list?titleId=817945')

{'page': 1}
https://comic.naver.com/webtoon/detail?titleId=817945&no=133
https://comic.naver.com/webtoon/detail?titleId=817945&no=132
https://comic.naver.com/webtoon/detail?titleId=817945&no=131
https://comic.naver.com/webtoon/detail?titleId=817945&no=130
https://comic.naver.com/webtoon/detail?titleId=817945&no=129
https://comic.naver.com/webtoon/detail?titleId=817945&no=128
https://comic.naver.com/webtoon/detail?titleId=817945&no=127
https://comic.naver.com/webtoon/detail?titleId=817945&no=126
https://comic.naver.com/webtoon/detail?titleId=817945&no=125
https://comic.naver.com/webtoon/detail?titleId=817945&no=124
https://comic.naver.com/webtoon/detail?titleId=817945&no=123
https://comic.naver.com/webtoon/detail?titleId=817945&no=122
https://comic.naver.com/webtoon/detail?titleId=817945&no=121
https://comic.naver.com/webtoon/detail?titleId=817945&no=120
https://comic.naver.com/webtoon/detail?titleId=817945&no=119
https://comic.naver.com/webtoon/detail?titleId=817945&no=118
https://comi